In [1]:
import (
    "bufio"
    "fmt"
    "strings"
    "strconv"
    "os"
    "math"
    "io/ioutil"
    "encoding/json"
)

type Packet struct {
    Version int
    Type int
    Value int
    SubPackets []*Packet
}

var VersionSum int = 0

func PrettyPrint(v interface{}) (err error) {
      b, err := json.MarshalIndent(v, "", "  ")
      if err == nil {
              fmt.Println(string(b))
      }
      return
}

func getBits(ch <-chan string, numBits int) uint64 {
    if len(ch) < numBits {
        return 0
    }
    var bStr string = ""
    for i := 0; i < numBits; i++ {
        bStr += <- ch
    }
    iVal,_ := strconv.ParseUint(bStr,2,32)
    return iVal
}

func loadData(file string) chan string{
    f, err := ioutil.ReadFile(file)
    rawData := strings.Split(string(f),"")
    ch := make(chan string, len(rawData)*4)
    for i,sVal := range rawData {
        intVal, _ := strconv.ParseUint(strings.TrimSuffix(sVal, "\n"), 16, 32)
        bStr := fmt.Sprintf("%04b",intVal)
        for _,v := range bStr {
            ch <- string(v)
        }
    }
    return ch
}

func getLiteral(ch <-chan string) int {
    tFlag := getBits(ch,1)
    valParts := []uint64{}
    for tFlag != 0 {
        valParts = append(valParts, getBits(ch,4))
        tFlag = getBits(ch,1)
    }
    valParts = append(valParts,getBits(ch,4))
    litVal := 0
    for i,v := range valParts {
        litVal += int(v << ((len(valParts)-1-i)*4))
    }
    return litVal
}

func getOperation(ch <-chan string) []*Packet {
    lenTypeID := getBits(ch,1)
    subPackets := []*Packet{}
    switch lenTypeID {
    case 0:
        tLen := int(getBits(ch,15))
        finished := len(ch) - tLen
        remaining := len(ch) - finished
        for remaining != 0 {
            packet := new(Packet)
            packet.Version = int(getBits(ch,3))
            packet.Type = int(getBits(ch,3))
            VersionSum += packet.Version
            if packet.Type == 4 {
                packet.Value = getLiteral(ch)
                remaining = len(ch) - finished
                subPackets = append(subPackets,packet)
            } else {
                packet.SubPackets = append(packet.SubPackets, getOperation(ch)...)
                remaining = len(ch) - finished
                subPackets = append(subPackets,packet)
            }
            remaining = len(ch) - finished
            if remaining < 8 {
                devNull := int(getBits(ch,remaining))
            }
            remaining = len(ch) - finished
        }
    case 1:
        stLen := len(ch)
        numSP := int(getBits(ch,11))
        for i := 0 ; i < numSP; i++ {
            packet := new(Packet)
            packet.Version = int(getBits(ch,3))
            packet.Type = int(getBits(ch,3))
            VersionSum += packet.Version
            if packet.Type == 4 {
                packet.Value = getLiteral(ch)
                subPackets = append(subPackets,packet)
            } else {
                packet.SubPackets = append(packet.SubPackets, getOperation(ch)...)
                subPackets = append(subPackets,packet)
            }
        }
    }
    return subPackets
}

// Part 1
func sumVersions (packet *Packet) int {
    total := packet.Version
    for i,v := range packet.SubPackets {
        total += sumVersions(v)
    }
    return total
}


// Part 2
func calculatePacketValue (packet *Packet) int {
    if len(packet.SubPackets) == 0 {
        return packet.Value
    }
    switch packet.Type {
    case 0: // sum
        for i,v := range packet.SubPackets {
            packet.Value += calculatePacketValue(v)
        }
    case 1: // product
        packet.Value = calculatePacketValue(packet.SubPackets[0])
        for i := 1; i < len(packet.SubPackets); i++ {
            packet.Value *= calculatePacketValue(packet.SubPackets[i])
        }
    case 2: // min
        packet.Value = int(^uint(0) >> 1)
        for i,v := range packet.SubPackets {
            tVal := calculatePacketValue(v)
            if tVal < packet.Value {
                packet.Value = tVal
            }
        }
    case 3: // max
        packet.Value = int(^uint(0)>>1)*(-1) - 1
        for i,v := range packet.SubPackets {
            tVal := calculatePacketValue(v)
            if tVal > packet.Value {
                packet.Value = tVal
            }
        }
    case 5: // first greater than second
        if calculatePacketValue(packet.SubPackets[0]) > calculatePacketValue(packet.SubPackets[1]) {
            packet.Value = 1
        }
    case 6: // first less than second
        if calculatePacketValue(packet.SubPackets[0]) < calculatePacketValue(packet.SubPackets[1]) {
            packet.Value = 1
        }
    case 7: // equal to
        if calculatePacketValue(packet.SubPackets[0]) == calculatePacketValue(packet.SubPackets[1]) {
            packet.Value = 1
        }
    }
    return packet.Value
}

packet := new(Packet)
ch := loadData("input_data")

packet.Version = int(getBits(ch,3))
packet.Type = int(getBits(ch,3))
VersionSum += packet.Version

switch packet.Type {
case 4: // outer packet is only one literal value
    litVal := getLiteral(ch)
    packet.Value = litVal
default: // outer packet is an operation
    packet.SubPackets = append(packet.SubPackets, getOperation(ch)...)   
}

//PrettyPrint(packet)
fmt.Println("Sum of version numbers:",sumVersions(packet))

Sum of version numbers: 891


28 <nil>

In [2]:
fmt.Println("Calculated value is:",calculatePacketValue(packet))

Calculated value is: 673042777597


34 <nil>